c_base2021_500d_150eV2

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
import numpy as np
import tensorflow as tf
path = r'..\\embeddings\\'
id_dict = {}
word_dict = {}

matrix = []
dimensions = 500
with open(path + "m_base2021_500d_100e_wc", 'r' , encoding="utf-8")  as f:
    lines = f.readlines()
    vocab_size = len(lines)
    
    matrix = np.zeros((vocab_size,dimensions),dtype=float)
    for line in lines:
        values = line.split()
        word = values[0].strip()
        id = len(id_dict)
        id_dict[word]=id
        word_dict[id] = word
        vector = np.asarray(values[1:], "double")
        matrix[id_dict[word],:] = vector
print(len(id_dict))

260377


In [3]:
def find_candidate(sim,id_a,id_a_star,id_b):
    myList = [id_a, id_b, id_a_star];
    myList.sort();
    
    if(myList[0] != 0):
        index1 = np.argmax(sim[:myList[0]])
    else:
        index1 = None
    sim2 = sim[myList[0]+1:myList[1]]
    if len(sim2) > 0:
        index2 = np.argmax(sim2)
        index2 +=  myList[0]+1
    else:
        index2 = None
    sim3 = sim[myList[1]+1:myList[2]]
    if len(sim3) > 0:
        index3 = np.argmax(sim3)
        index3 +=  myList[1]+1
    else:
        index3 = None
    if(myList[2]+1 < len(sim)):
        index4 = np.argmax(sim[myList[2]+1:])
        index4 +=  myList[2]+1
    else:
        index4 = None
    myList = [index1, index2, index3,index4];
    values = []
    myList2 = []
    for index in myList:
        if index != None:
            values.append(sim[index])
            myList2.append(index)
    index = myList2[np.argmax(values)]
    return index

In [4]:
matrix_normalized = tf.nn.l2_normalize(matrix,axis = 1)

In [5]:
def find_nearest_k(searched_word,k):
    list = []
    id = id_dict[searched_word]
    searched_vector = matrix_normalized[id,:] 
    
    for word in id_dict:
        word_weights = matrix_normalized[id_dict[word]]
        loss = tf.tensordot(word_weights,searched_vector,axes = 1).numpy()
        list = insert(list,(word,loss))
        if len(list) > k:
            list = list[0:k+1]
    return list[0:k]

# Function to insert element
def insert(list, tuple):
    (word,n) = tuple
    if(len(list) == 0):
        list = [(word,n)]
    # Searching for the position
    for i in range(len(list)):
        (word_i,n_i) = list[i]
        if n_i < n:
            index = i
            break
      
    # Inserting n in the list
    list = list[:i] + [(word,n)] + list[i:]
    return list


print(find_nearest_k('batman',6))#this should produce one, for the same vectors
print(find_nearest_k('hogwarts',6))
print(find_nearest_k('turing',6))
print(find_nearest_k('florida',6))
#print(find_nearest_k('object-oriented',6))
print(find_nearest_k('dancing',6))

[('batman', 0.9999999999999996), ('superman', 0.6855094916939756), ('joker', 0.6363511918428321), ('catwoman', 0.6361917217258489), ('kozluk', 0.5977651794775573), ('nightwing', 0.5886694421917177)]
[('hogwarts', 1.0000000000000004), ('rameswaram', 0.5302505455777797), ('humsafar', 0.5128881415105725), ('dumbledore', 0.511255774830167), ('sinhagad', 0.5074298043666995), ('express', 0.5066376430301401)]
[('turing', 1.0), ('deterministic', 0.5605650141247839), ('machine', 0.5177206244686243), ('machines', 0.4869612617024984), ('nondeterministic', 0.47225329551892814), ('replicating', 0.4701054526555373)]
[('hogwarts', 1.0000000000000004), ('rameswaram', 0.5302505455777797), ('humsafar', 0.5128881415105725), ('dumbledore', 0.511255774830167), ('sinhagad', 0.5074298043666995), ('express', 0.5066376430301401)]
[('florida', 0.9999999999999999), ('tallahassee', 0.7063361671827202), ('gainesville', 0.6989361200428146), ('jacksonville', 0.6581077242727743), ('gators', 0.6536703951703233), ('mia

KeyError: 'object-oriented'

## 3cosAdd 

In [6]:

def three_cos_predict_np(a,a_star,b,b_star):
    if( a in id_dict and b in id_dict and a_star in id_dict and b_star in id_dict):
        pass
    else:
        return None
    id_a = id_dict[a]
    id_b = id_dict[b]
    id_a_star = id_dict[a_star]
    id_b_star = id_dict[b_star]#remove this after testing
    weight_a = matrix[id_a,:]
    weight_b = matrix[id_b,:]
    weight_a_star = matrix[id_a_star,:]
    direction = weight_b + ( weight_a_star - weight_a)
    direction /= np.linalg.norm(direction)
    
    sim = tf.tensordot(tf.convert_to_tensor(matrix_normalized),tf.convert_to_tensor(direction),axes = 1)
    index = find_candidate(sim,id_a,id_a_star,id_b)
    return word_dict[index]

# 3cosAdd normalised befor arithmetic

In [7]:
def three_cos_predict_np_norm(a,a_star,b,b_star):
    if( a in id_dict and b in id_dict and a_star in id_dict and b_star in id_dict):
        pass
    else:
        return None
    id_a = id_dict[a]
    id_b = id_dict[b]
    id_a_star = id_dict[a_star]
    id_b_star = id_dict[b_star]#remove this after testing
    weight_a = matrix_normalized[id_a,:]
    weight_b = matrix_normalized[id_b,:]
    weight_a_star = matrix_normalized[id_a_star,:]
    direction = weight_b + ( weight_a_star - weight_a)
    direction /= np.linalg.norm(direction)
    
    sim = tf.tensordot(tf.convert_to_tensor(matrix_normalized),tf.convert_to_tensor(direction),axes = 1)
    index = find_candidate(sim,id_a,id_a_star,id_b)
    return word_dict[index]

## 3cos Mult

In [8]:
def sim_01(matrix,weight):
    tmp = tf.tensordot(matrix,weight,axes=1)
    tmp = (tmp+1.0)/2.0
    return tmp

def predict_three_cos_mult(a,a_star,b,b_star):
    if( a in id_dict and b in id_dict and a_star in id_dict and b_star in id_dict):
        pass
    else:
        print(a in id_dict)
        print(a_star in id_dict)
        print(b in id_dict)
        print(b_star in id_dict)
        return None
    
    id_a = id_dict[a]
    id_b = id_dict[b]
    id_a_star = id_dict[a_star]
    
    weight_a = matrix_normalized[id_a,:]
    weight_b = matrix_normalized[id_b,:]
    weight_a_star = matrix_normalized[id_a_star,:]
    
    nominator = sim_01(matrix_normalized,weight_a_star) * sim_01(matrix_normalized,weight_b) 
    denominator = sim_01(matrix_normalized,weight_a) + 0.0001
    sim = nominator / denominator
    index = find_candidate(sim,id_a,id_a_star,id_b)
    return word_dict[index]

# Test all

In [9]:
count_sem_questions =  0
count_syn_questions =  0

count_sem_sucess      = [0,0,0]
count_syn_sucess      = [0,0,0]

errors = []


files = os.listdir('.\\datasets\\question-data')
for idx,file_name in enumerate(files):
    print(file_name)
    with open('.\\datasets\\question-data\\'+file_name,'r') as file:
        lines = file.readlines()
        local_sucesses    = [0,0,0]
        local_tasks_count = 0
        
        for line_id,line in enumerate(lines):
            if (line_id % 100 == 0):
                print(line_id)
            a,a_star,b,b_star = line.split(" ")
            b_star = b_star.strip()
            #predicted_3cos_add      = three_cos_predict_np(a,a_star,b,b_star)
            predicted_3cos_add_norm = three_cos_predict_np_norm(a,a_star,b,b_star)
            predicted_3cos_mult     = predict_three_cos_mult(a,a_star,b,b_star)
            #print(a,a_star,b,b_star)
            #print(predicted_3cos_add,predicted_3cos_add_norm,predicted_3cos_mult)
            if(idx < 5):#first 5 are sem
                count_sem_questions += 1
                local_tasks_count   += 1
                
                #if predicted_3cos_add      == b_star:
                #    count_sem_sucess[0] += 1
                if predicted_3cos_add_norm == b_star:
                    local_sucesses[1] += 1
                    count_sem_sucess[1] += 1
                    
                if  predicted_3cos_mult    == b_star:
                    local_sucesses[2] += 1
                    count_sem_sucess[2] += 1
            else:
                count_syn_questions += 1
                local_tasks_count   += 1
                
                #if predicted_3cos_add      == b_star:
                #    count_syn_sucess[0] += 1
                if predicted_3cos_add_norm == b_star:
                    local_sucesses[1] += 1
                    count_syn_sucess[1] += 1
                    
                if  predicted_3cos_mult    == b_star:
                    local_sucesses[2] += 1
                    count_syn_sucess[2] += 1
        print(local_sucesses[0]/float(local_tasks_count))
        print(local_sucesses[1]/float(local_tasks_count))
        print(local_sucesses[2]/float(local_tasks_count))
        print('-------------------')

print('semantical')

print(count_sem_sucess[0]/float(count_sem_questions))
print(count_sem_sucess[1]/float(count_sem_questions))
print(count_sem_sucess[2]/float(count_sem_questions))
        
print('syntactical')
print(count_syn_sucess[0]/float(count_sem_questions))
print(count_syn_sucess[1]/float(count_sem_questions))
print(count_syn_sucess[2]/float(count_sem_questions))

print('overall')
print((count_syn_sucess[0]+count_sem_sucess[0])/float(count_sem_questions+count_sem_questions))
print((count_syn_sucess[1]+count_sem_sucess[1])/float(count_sem_questions+count_sem_questions))
print((count_syn_sucess[2]+count_sem_sucess[2])/float(count_sem_questions+count_sem_questions))

capital-common-countries.txt
0
100
200
300
400
500
0.0
0.9525691699604744
0.9565217391304348
-------------------
capital-world.txt
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
0.0
0.9029619805481874
0.9146772767462422
-------------------
city-in-state.txt
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
0.0
0.7065261451155249
0.7259829752736117
-------------------
currency.txt
0
True
True
True
False
True
True
True
False
True
True
True
False
100
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
200
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
300
True
True
True
False
True
True
True
False
True
True
True
False
400
True
True
True
False
True
True
True
False
True
True
True
False
True
Tru

TypeError: 'int' object is not subscriptable

In [19]:
print('semantical')

print(count_sem_sucess[0]/float(count_sem_questions))
print(count_sem_sucess[1]/float(count_sem_questions))
print(count_sem_sucess[2]/float(count_sem_questions))
        
print('syntactical')
print(count_syn_sucess[0]/float(count_sem_questions))
print(count_syn_sucess[1]/float(count_sem_questions))
print(count_syn_sucess[2]/float(count_sem_questions))

print('overall')
print((count_syn_sucess[0]+count_sem_sucess[0])/float(count_sem_questions+count_sem_questions))
print((count_syn_sucess[1]+count_sem_sucess[1])/float(count_sem_questions+count_sem_questions))
print((count_syn_sucess[2]+count_sem_sucess[2])/float(count_sem_questions+count_sem_questions))

semantical
0.0
0.7644604803247266
0.7805840568271507
syntactical
0.0
0.5415492163716316
0.5566580223249521
overall
0.0
0.6530048483481791
0.6686210395760515


In [20]:
import winsound
winsound.Beep(440, 500)

Spearman Rank

In [22]:
with open(r'.\datasets\wordsim353_sim_rel\wordsim_similarity_goldstandard.txt') as file:

    losses = []
    scores = []

    lines = file.readlines()
    sum_diff = 0
    for line in lines:
        if(len(line) <= 1):
            pass

        print("line" + line)
        word1, word2, score10 = line.split()
        score10 = float(score10)
        try:
            print(word1,word2)
            id1 = id_dict[word1]
            id2 = id_dict[word2]
            print(id1)
            print(id2)
            vector1 = matrix_normalized[id1,:]
            vector2 = matrix_normalized[id2,:]
            #print(vector1)
            #print(vector2)
            loss = sim_01(vector1,vector2)
        except: 
            loss = 0
        print(str(loss),str(score10/10.0))
        sum_diff += abs((score10/10.0) - loss)
        losses.append(loss)
        scores.append(score10/10.0)
    print('Average Distance between prediction and hand assigned is ' + str(sum_diff / len(lines)))

linetiger	cat	7.35

tiger cat
174824
1460
tf.Tensor(0.7071245157543795, shape=(), dtype=float64) 0.735
linetiger	tiger	10.00

tiger tiger
174824
174824
tf.Tensor(0.9999999999999998, shape=(), dtype=float64) 1.0
lineplane	car	5.77

plane car
237142
260365
tf.Tensor(0.6810725545591447, shape=(), dtype=float64) 0.577
linetrain	car	6.31

train car
225926
260365
tf.Tensor(0.7627709778870322, shape=(), dtype=float64) 0.631
linetelevision	radio	6.77

television radio
173375
82259
tf.Tensor(0.8049847641501349, shape=(), dtype=float64) 0.6769999999999999
linemedia	radio	7.42

media radio
127720
82259
tf.Tensor(0.7209574055297929, shape=(), dtype=float64) 0.742
linebread	butter	6.19

bread butter
105268
70044
tf.Tensor(0.7996707250754953, shape=(), dtype=float64) 0.619
linecucumber	potato	5.92

cucumber potato
188967
37462
tf.Tensor(0.707543556367068, shape=(), dtype=float64) 0.592
linedoctor	nurse	7.00

doctor nurse
114009
219162
tf.Tensor(0.7162988582902445, shape=(), dtype=float64) 0.7
linepr

linecup	food	5.00

cup food
180411
69227
tf.Tensor(0.5869379888915154, shape=(), dtype=float64) 0.5
linejournal	association	4.97

journal association
237304
52093
tf.Tensor(0.7012044452913561, shape=(), dtype=float64) 0.497
linestreet	children	4.94

street children
162693
169337
tf.Tensor(0.6551885253553931, shape=(), dtype=float64) 0.49400000000000005
linecar	flight	4.94

car flight
260365
24255
tf.Tensor(0.6433280228623885, shape=(), dtype=float64) 0.49400000000000005
linespace	chemistry	4.88

space chemistry
209979
16373
tf.Tensor(0.6101573998305757, shape=(), dtype=float64) 0.488
linesituation	conclusion	4.81

situation conclusion
62215
212997
tf.Tensor(0.685712727401267, shape=(), dtype=float64) 0.481
lineword	similarity	4.75

word similarity
3597
223717
tf.Tensor(0.6550102264147535, shape=(), dtype=float64) 0.475
linepeace	plan	4.75

peace plan
117566
142553
tf.Tensor(0.6576535031632895, shape=(), dtype=float64) 0.475
lineconsumer	energy	4.75

consumer energy
193006
231529
tf.Ten

tf.Tensor(0.6040289396374481, shape=(), dtype=float64) 0.092
linesugar	approach	0.88

sugar approach
59245
248049
tf.Tensor(0.5631224029405828, shape=(), dtype=float64) 0.088
linerooster	voyage	0.62

rooster voyage
159257
56156
tf.Tensor(0.5333960458437514, shape=(), dtype=float64) 0.062
linenoon	string	0.54

noon string
149540
82565
tf.Tensor(0.5788411474762858, shape=(), dtype=float64) 0.054000000000000006
linechord	smile	0.54

chord smile
85801
241779
tf.Tensor(0.5881126615299979, shape=(), dtype=float64) 0.054000000000000006
lineprofessor	cucumber	0.31

professor cucumber
45172
188967
tf.Tensor(0.5460122194692313, shape=(), dtype=float64) 0.031
lineking	cabbage	0.23

king cabbage
61365
18555
tf.Tensor(0.5240856965701292, shape=(), dtype=float64) 0.023
Average Distance between prediction and hand assigned is tf.Tensor(0.22140774751883804, shape=(), dtype=float64)


In [23]:
from scipy import stats
stats.spearmanr(losses, scores)

SpearmanrResult(correlation=0.6221037410592645, pvalue=3.903560797855815e-23)